<h1 style="text-align: center">
Deep Learning HW3 </br>
Q3
</h1>


Here we aim to generate poems like ferdowsi by fin-tuning GPT2

In [1]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import Dataset  # this is the pytorch class import
torch.manual_seed(42)
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer,TextDataset,DataCollatorForLanguageModeling,Trainer,TrainingArguments,AutoModelWithLMHead,AutoConfig

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [4]:
i = 0
poems = []
file = open("/content/drive/MyDrive/Deep_Learning/HW4/Q3/ferdousi.txt", "r")
while True:
  content = file.readline()
  if not content:
    break
  elif i>1:
    content = content.strip()
    poems.append(content)
  i=i+1
file.close()

In [5]:
poems[:10]

['به نام خداوند جان و خرد',
 'کزین برتر اندیشه برنگذرد',
 'خداوند نام و خداوند جای',
 'خداوند روزی ده رهنمای',
 'خداوند کیوان و گردان سپهر',
 'فروزنده ماه و ناهید و مهر',
 'ز نام و نشان و گمان برترست',
 'نگارندهٔ بر شده پیکرست',
 'به بینندگان آفریننده را',
 'نبینی مرنجان دو بیننده را']

In [6]:
poems = np.array(poems)
poems.shape

(99217,)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Preprossing data and dataSplit

In [8]:
input_poems = poems[list(range(0,len(poems),2))]
output_poems = poems[list(range(1,len(poems),2))]

In [9]:
split_ratio = 0.8
split_index = int(len(input_poems) * split_ratio)

train_input_poems = input_poems[:split_index]
train_output_poems = output_poems[:split_index]

test_input_poems = input_poems[split_index:]
test_output_poems = output_poems[split_index:]

print(train_input_poems[:10])
print(train_output_poems[:10])

['به نام خداوند جان و خرد' 'خداوند نام و خداوند جای'
 'خداوند کیوان و گردان سپهر' 'ز نام و نشان و گمان برترست'
 'به بینندگان آفریننده را' 'نیابد بدو نیز اندیشه راه'
 'سخن هر چه زین گوهران بگذرد' 'خرد گر سخن برگزیند همی'
 'ستودن نداند کس او را چو هست' 'خرد را و جان را همی سنجد اوی']
['کزین برتر اندیشه برنگذرد' 'خداوند روزی ده رهنمای'
 'فروزنده ماه و ناهید و مهر' 'نگارندهٔ بر شده پیکرست'
 'نبینی مرنجان دو بیننده را' 'که او برتر از نام و از جایگاه'
 'نیابد بدو راه جان و خرد' 'همان را گزیند که بیند همی'
 'میان بندگی را ببایدت بست' 'در اندیشهٔ سخته کی گنجد اوی']


In [10]:
print(test_input_poems[:10])
print(test_output_poems[:10])

['پس پردهٔ او بسی درخترست' 'پرستار زاده نیاید به کار'
 'نگر تا کدامست با شرم و داد' 'نبیره جهاندار فغفور چین'
 'اگر گوهرتن بود با نژاد' 'چوبشنید مهران ستاد این ز شاه'
 'برفت از بر گاه گیتی فروز' 'به خاقان چین آگهی شد که شاه'
 'چوآمد به نزدیک خاقان چین' 'جهانجوی چون دید بنواختش']
['که با فر و بالا و با افسرست' 'اگر چند باشد پدر شهریار'
 'به مادر که دارد ز خاتون نژاد' 'ز پشت سپهدار خاقان چین'
 'جهان زو شود شاد او نیز شاد' 'بسی آفرین کرد بر تاج و گاه'
 'به فرخنده فال و بخرداد روز' 'فرستاده مهران ستاد و سپاه'
 'زمین را ببوسید و کرد آفرین' 'یکی نامور جایگه ساختش']


## tokenizing and creating dataloaders

In [11]:
model_name = "HooshvareLab/gpt2-fa"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    bos_token='<s>',
    eos_token='</s>',
    pad_token='<pad>',
    unk_token='<unk>'
)
tokenizer.add_special_tokens({
    "bos_token": '</s>',
    "eos_token": '</s>',
    "pad_token": '<pad>',
    "unk_token": '<unk>'
})

config = AutoConfig.from_pretrained(
    model_name,
    bos_token_id=tokenizer("<s>")["input_ids"][0],
    eos_token_id=tokenizer("</s>")["input_ids"][0],
    pad_token_id=tokenizer("<pad>")["input_ids"][0],
    unk_token_id=tokenizer("<unk>")["input_ids"][0],
)

print(tokenizer.encode("سلام بر شما"))
print(tokenizer.encode("<s>"))
print(tokenizer.encode("</s>"))
print(tokenizer.encode("<pad>"))
print(tokenizer.encode("<|startoftext|>"))
print(tokenizer.encode("<sep>"))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/875k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.75M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/14.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/104 [00:00<?, ?B/s]

[8906, 327, 512]
[0]
[2]
[1]
[6]
[9]


In [12]:
class PoemDataset(Dataset):
    def __init__(self, input_poems,output_poems, tokenizer, max_length=64):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.output_ids = []
        self.attn_masks = []

        for input,output in zip(input_poems,output_poems):
            encodings_dict_in = tokenizer('<s>' + input + '</s>',
                                       truncation=True,
                                       max_length=max_length,
                                       padding="max_length")
            encodings_dict_out = tokenizer('<s>' + output + '</s>',
                                       truncation=True,
                                       max_length=max_length,
                                       padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict_in['input_ids']))
            self.output_ids.append(torch.tensor(encodings_dict_out['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict_out['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx],self.output_ids[idx], self.attn_masks[idx]

In [13]:
max_seq = 15
dataset_train = PoemDataset(train_input_poems,train_output_poems, tokenizer, max_length=max_seq)
dataset_test = PoemDataset(test_input_poems, test_output_poems, tokenizer,max_length=max_seq)

f'There are {len(dataset_train)} samples for training, and {len(dataset_test)} samples for test'

'There are 39687 samples for training, and 9921 samples for test'

In [14]:
def arr_to_srt(token):
  return token.split('</s>')[0][3:]

In [15]:
# print(dataset_train[0][0])
print(arr_to_srt(tokenizer.decode(dataset_train[0][0])))
# print(dataset_train[0][1])
print(arr_to_srt(tokenizer.decode(dataset_train[0][1])))

به نام خداوند جان و خرد
کزین برتر اندیشه برنگذرد


In [16]:
train_dataloader = DataLoader(dataset_train, batch_size=4, shuffle=True)
test_dataloader = DataLoader(dataset_test, batch_size=4, shuffle=False)

In [17]:
input , output , mask = next(iter(train_dataloader))
print(tokenizer.decode(input[0]))
print(arr_to_srt(tokenizer.decode(input[0])))
print(arr_to_srt(tokenizer.decode(output[0])))
print(mask[0])

<s>هزار اشتر بختی سرخ موی</s><pad><pad><pad><pad><pad><pad><pad><pad>
هزار اشتر بختی سرخ موی
بنه بر نهادند با رنگ و بوی
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0])


## Loading pretrain model and training with crossentropy

In [18]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [19]:
tokenizer(train_input_poems[1])

{'input_ids': [12090, 595, 561, 293, 6733, 798], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [20]:
model = GPT2LMHeadModel.from_pretrained("HooshvareLab/gpt2-fa",config=config).to(device)

pytorch_model.bin:   0%|          | 0.00/485M [00:00<?, ?B/s]

In [ ]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(42001, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=42001, bias=False)
)


In [ ]:
batch = next(iter(train_dataloader))
batch[0].shape

torch.Size([4, 15])

In [25]:
def print_input_output(input,output):
  print(f'input:  {input}')
  print(f'output:  {output[:len(input)]} , {output[len(input):]}')

test before fine tuning:

In [ ]:
input , output , mask = next(iter(train_dataloader))
# type(model(input))
model.eval()
sample_outputs = model.generate(
    input_ids=input.to(device),
    # bos_token_id=random.randint(1, len(tokenizer.get_vocab())),
    attention_mask=torch.ones_like(input).to(device),
    do_sample=False,
    top_k=50,
    max_length=25,
    num_beams=5,
    no_repeat_ngram_size=1,
    num_return_sequences=1,
    # top_p=0.95,
)
for i, sample_output in enumerate(sample_outputs):
    gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
    gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
    gen_sample_output = gen_sample_output.replace("<s>", "")
    gen_sample_output = gen_sample_output.replace("</s>", "")
    gen_sample_output = gen_sample_output.replace("<sep>", "\n")
    gen_sample_output = gen_sample_output.replace("<pad>", " ")
    gen_sample_input = tokenizer.decode(input[i], skip_special_tokens=True)
    print_input_output(gen_sample_input,gen_sample_output)
    # print(f'Example input: {gen_sample_input}')
    # print(f'Example output: {gen_sample_output}\n\n')

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


input:  نه هرکس که شد پادشاهی ببرد
output:  نه هرکس که شد پادشاهی ببرد ,        جات را از دست می‌دهد. در این
input:  و گر سوی ایران براند سپاه
output:  و گر سوی ایران براند سپاه ,        دین را از دست می‌دهند. در این
input:  که برگشت زین کینه افراسیاب
output:  که برگشت زین کینه افراسیاب ,         جات را از دست می‌دهد. در این
input:  چنان برگذشتند هر سه سوار
output:  چنان برگذشتند هر سه سوار ,       انیست که از قافله عقب مانده‌اند. در


### Training

In [ ]:
import random
import time
import datetime
from tqdm import tqdm

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

optimizer = AdamW(
    model.parameters(),
    lr=5e-4,
    eps=1e-8
)

epochs = 3
warmup_steps = 1e2

total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
sample_every = 300
total_t0 = time.time()


def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))


training_stats = []
model = model.to(device)
for epoch_i in tqdm(range(0, epochs), position=0):
    print(f'Beginning epoch {epoch_i + 1} of {epochs}')
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), position=0):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)
        b_masks = batch[2].to(device)
        model.zero_grad()
        outputs = model(b_input_ids, labels=b_labels, attention_mask=b_masks, token_type_ids=None)
        loss = outputs[0]
        batch_loss = loss.item()
        total_train_loss += batch_loss

        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)

            print(f'\n Batch {step} of {len(train_dataloader)}. Loss:{batch_loss}. Time:{elapsed}')
            model.eval()
            input , output , mask = next(iter(train_dataloader))
            sample_outputs = model.generate(
                input_ids=input.to(device),
                # bos_token_id=random.randint(1, len(tokenizer.get_vocab())),
                attention_mask=torch.ones_like(input).to(device),
                do_sample=False,
                top_k=50,
                max_length=25,
                num_beams=5,
                no_repeat_ngram_size=1,
                num_return_sequences=1,
            )
            for i, sample_output in enumerate(sample_outputs):
                s_output = tokenizer.decode(sample_output, skip_special_tokens=False)
                s_output = s_output.replace("<|startoftext|>", "\n")
                s_output = s_output.replace("<s>", "")
                s_output = s_output.replace("</s>", "")
                s_output = s_output.replace("<sep>", "\n")
                s_output = s_output.replace("<pad>", "")
                s_input = tokenizer.decode(input[i], skip_special_tokens=True)
                print()
                print_input_output(s_input,s_output)

            model.train()

        loss.backward()
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)


    print(f'\n Average Training Loss: {avg_train_loss}. Epoch time: {training_time}\n')


    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0


    for batch in tqdm(dataset_test, total=len(dataset_test), position=0):

        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)
        b_masks = batch[2].to(device)

        with torch.no_grad():

            outputs = model(b_input_ids, attention_mask=b_masks, labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(dataset_test)

    validation_time = format_time(time.time() - t0)

    print(f'\n Validation loss: {avg_val_loss}. Validation Time: {validation_time} \n')



    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print(f'Total training took {format_time(time.time()-total_t0)}')

  0%|          | 0/3 [00:00<?, ?it/s]

Beginning epoch 1 of 3


  3%|▎         | 300/9922 [00:25<11:23, 14.07it/s]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



Batch 300 of 9922. Loss:3.4098267555236816. Time:0:00:25

input:  می و هرچ خوردی ترا نوش باد
output:  می و هرچ خوردی ترا نوش باد , نداستستاند ز بر رویید ماه کم

input:  وزانجا سوی کاخ رفتند باز
output:  وزانجا سوی کاخ رفتند باز , نداست و برستاندیدش ز کم

input:  تو گفتی که برشد به گیتی بخار
output:  تو گفتی که برشد به گیتی بخار , نداست ویدستاند از ز ماه کم

input:  چهارم هران پیر کز کارکرد
output:  چهارم هران پیر کز کارکرد , نداست و برستاند زید ماه کم


  6%|▌         | 600/9922 [00:47<12:28, 12.45it/s]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...


Batch 9900 of 9922. Loss:3.0760715007781982. Time:0:12:39

input:  و گر نشنود بودنیها درست
output:  و گر نشنود بودنیها درست ,  وندستش رایم است تست بش ب

input:  شدند انجمن دیو بسیار مر
output:  شدند انجمن دیو بسیار مر ,  و کمر زر سر بر کران زعفراناورانوار گران

input:  چو آگاه شد زان سخن هفتواد
output:  چو آگاه شد زان سخن هفتواد ,  یاد داداد و ز باد را او بود کرد

input:  پی کین نهان گردد از روی بوم
output:  پی کین نهان گردد از روی بوم ,  و بهم را او بران تو منیم


100%|██████████| 9922/9922 [12:40<00:00, 13.04it/s]



Average Training Loss: 3.2454409823978025. Epoch time: 0:12:41



100%|██████████| 3/3 [45:28<00:00, 909.49s/it]


Validation loss: 3.4220286226839733. Validation Time: 0:03:04

Total training took 0:45:28


## test and belu score

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Deep_Learning/HW4/Q3/ferdosi_generator.pth')

In [21]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Deep_Learning/HW4/Q3/ferdosi_generator.pth'))

<All keys matched successfully>

In [30]:
from nltk.translate.bleu_score import sentence_bleu
import nltk
nltk.download('punkt')
def generate_poem(model,data_loader_itertor):
  input , output , mask = next(data_loader_itertor)
  # type(model(input))
  model.eval()
  sample_outputs = model.generate(
      input_ids=input.to(device),
      # bos_token_id=random.randint(1, len(tokenizer.get_vocab())),
      attention_mask=torch.ones_like(input).to(device),
      do_sample=False,
      top_k=50,
      max_length=25,
      num_beams=5,
      no_repeat_ngram_size=1,
      num_return_sequences=1,
      # top_p=0.95,
  )
  for i, sample_output in enumerate(sample_outputs):
      s_output = tokenizer.decode(sample_output, skip_special_tokens=False)
      s_output = s_output.replace("<|startoftext|>", "\n")
      s_output = s_output.replace("<s>", "")
      s_output = s_output.replace("</s>", "")
      s_output = s_output.replace("<sep>", "\n")
      s_output = s_output.replace("<pad>", " ")
      s_input = tokenizer.decode(input[i], skip_special_tokens=True)
      print_input_output(s_input,s_output)
      out = tokenizer.decode(output[i],skip_special_tokens=True)
      bleu_score = BLEU(out,s_output)#gen_sample_output[len(gen_sample_input):].strip()) # 1-gram
      print(bleu_score)
def BLEU(reference,output):
    # print(reference)
    # print(output)
    reference_words = nltk.word_tokenize(reference)
    # print(reference_words)
    output_words = nltk.word_tokenize(output)
    # print(output_words)
    bleu_score = sentence_bleu([reference_words], output_words,weights=(1,0,0,0),)
    return bleu_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
test_itertor = iter(test_dataloader)

In [33]:
generate_poem(model,test_itertor)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


input:  اگر گوهرتن بود با نژاد
output:  اگر گوهرتن بود با نژاد ,         یاد داد و شاداد ز را او به باد
0.07142857142857141
input:  چوبشنید مهران ستاد این ز شاه
output:  چوبشنید مهران ستاد این ز شاه ,        گاه کلاه سپاه ماه و بخت تخت راهان به
0.13333333333333333
input:  برفت از بر گاه گیتی فروز
output:  برفت از بر گاه گیتی فروز ,      ان وندش او به بد بود ز سر
0.07142857142857141
input:  به خاقان چین آگهی شد که شاه
output:  به خاقان چین آگهی شد که شاه ,        گاه کلاه سپاه ماه وان زمینگاه خواه راه
0.06666666666666667


In [34]:
generate_poem(model,test_itertor)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


input:  چوآمد به نزدیک خاقان چین
output:  چوآمد به نزدیک خاقان چین ,         زمین کینان ز و بر شاهگاه کلاه آفرین
0.23076923076923078
input:  جهانجوی چون دید بنواختش
output:  جهانجوی چون دید بنواختش ,       شان خواندنداندندند خواستندپرداختند تا و برش خواسته
0
input:  ازان کارخاقان پراندیشه گشت
output:  ازان کارخاقان پراندیشه گشت ,      راش بر وند او بهید شنید کشید
0
input:  سخنهای نوشین روان برگشاد
output:  سخنهای نوشین روان برگشاد ,         یاد داد ز واد شاد باد راان آزاد
0.16666666666666669


In [35]:
generate_poem(model,test_itertor)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


input:  بدو گفت کین شاه نوشین روان
output:  بدو گفت کین شاه نوشین روان ,        ایست تو من و بن این را از سخن
0.06666666666666667
input:  یکی دختری داد باید بدوی
output:  یکی دختری داد باید بدوی ,          روی و بوی جویگوی اوی به را او باد
0.07142857142857141
input:  تو را در پس پرده یک دخترست
output:  تو را در پس پرده یک دخترست ,      انش اوند و بود دست کس از بر
0.06666666666666667
input:  مرا آرزویست از مهر اوی
output:  مرا آرزویست از مهر اوی ,         روی جوی بوی و را او تو من جان باد
0.13333333333333333


In [ ]:
reference = [
    'سلام اینجا در حالی تخریب است نزدیک نشوید'.split(),
]
candidate = 'سلام اونجا داره خراب میشود نزدیک نشوید'.split()
print('Individual 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 0, 1)))
score = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))
print(score)

Individual 1-gram: 0.371519
Individual 2-gram: 0.144480
Individual 3-gram: 0.000000
Individual 4-gram: 0.000000
6.684949609530091e-155


thats why i use 1-gram

In [ ]:
!pip install lmppl

In [ ]:
import lmppl
scorer = lmppl.LM('gpt2')